So far, only the use of first-order derivatives has been discussed in this chapter. However, the process with first-order derivatives can be slow with some error surfaces. The problem is caused by the nature of <font color='red'>curvature</font> (i.e., <font color='blue'>second-order derivatives</font>, <font color='orange'>changing gradient</font>), where the first-order gradient does not contain the information needed to control the size of the update.

<font color='red'>In general, second-order methods approximate the local loss surface with a quadratic bowl, which is more accurate than the linear approximation.</font> Some second-order methods like the *Newton-methods* require exactly one iteration in order to find the local optimal solution for a quadratic surface. Of course, the loss surface of neural method is typically not quadratic. Nevertheless, the approximation is often good enough that gradient-descent methods are **greatly accelerated** at least in cases where the change in the gradient is not too sudden or drastic.


# Second-Order derivatives

Consider a parameter vector $W = (w_1 \cdots w_d)^T$, which is expressed as a column vector. Its gradient (i.e. first-order derivatives vector) of the loss function $L(W)$ is denoted by

$$G =\nabla L(W)= \left(\frac{\partial L(W)}{\partial w_1} \cdots \frac{\partial L(W)}{\partial w_d}\right)^T$$

And the second-order derivatives of the loss function are of the following form

$$H =\nabla \nabla L(W)= \begin{bmatrix}
\frac{\partial^2 L(W)}{\partial w_1\partial w_1} &\frac{\partial^2 L(W)}{\partial w_1\partial w_2} &\cdots &\frac{\partial^2 L(W)}{\partial w_1\partial w_d}\\
\frac{\partial^2 L(W)}{\partial w_2\partial w_1} &\frac{\partial^2 L(W)}{\partial w_2\partial w_2} &\cdots &\frac{\partial^2 L(W)}{\partial w_2\partial w_d}\\
\vdots &\vdots &\ddots &\vdots\\
\frac{\partial^2 L(W)}{\partial w_d\partial w_1} &\frac{\partial^2 L(W)}{\partial w_d\partial w_2} &\cdots &\frac{\partial^2 L(W)}{\partial w_d\partial w_d}\\
\end{bmatrix}$$

which is called *Hessian matrix*.

## Quadratic approximation and Newton update
One can write a quadratic approximation of the loss function in the vicinity of parameter vector $W_0$ by using the following Taylor expansion

$$L(W) \approx L(W_0) + (W-W_0)^T[\nabla L(W_0)]+\frac{1}{2}(W-W_0)^T H (W- W_0)$$

Where

- $W_0$ is the current parameter vector.
- $W$ is the vincinity of $W_0$.
- $\nabla L(W_0)$ is the gradient computed at $W_0$.
- $H$ is the Hessian computed at $W_0$.

For a quadratic loss function, the loss surface is a bowl, and we can use the Newton update to reach the bottom of the bowl in a single step.
$$\begin{align*}
\nabla L(W^*) &= \nabla \Big\{ L(W_0) + (W^*-W_0)^T[\nabla L(W_0)]+\frac{1}{2}(W^*-W_0)^T H (W^*- W_0) \Big\}\\
0 &= \nabla L(W_0) + H(W^*-W_0)\\
\Rightarrow W^* &= W_0 - H^{-1}[\nabla L(W_0)]
\end{align*}$$

However, in most cases, the loss function is more complicated, so that Newton method without learning rate can easily lead to gradient explosion, which does not meet our needs.

## Improvement from first-order to second-order

Comparing to the steepest-gradient descent, the second-order method (Newton update) premultiplies the gradient with the inverse of the Hessian. This premultiplication plays a key role in changing the direction of the steepest-graident towards low-curvature directions, because the Hessain matrix comprises the changing rate of every directions for each element in the gradient. This property can greatly accelerate the learning procedure and keep it from suffering oscillation in valley.

Also, the learning rate is already incorporated into the Hessian matrix implicitly, so that we don't need to determine the learning rate during the learning process.




## Limitation

In most large-scale neural network settings, the Hessian is too large to store or compute explicitly. It is not uncommon to have neural networks with millions of parameters. Trying to compute the inverse of a $10^6\times 10^6$ Hessian matrix is impractical with the computational power today. In fact, it is difficult to even compute the Hessian, let alone invert it!

Therefore, many approximations and variations of the Newton method have been developed.

-------------

# Line search along eigenvectors

An intuitive perspective for finding the optimum vector of the bowl given by a Hessian is to make the best (projected) gradient-descent step along each eigenvector so as to minimize the loss function. The best movement along a particular diretion is done using line search to select the step size.

------------------

# Conjugate Gradients
Traditional <a href='https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf'>conjugate gradients</a> algorithm is for solving the linear algebra problem, which is equivalent to minimizing quadratic functions. It has the procedure as follows.

|Step|Pseudocode|Description|
|----|----------|-----------|
|<img width=30/>|<img width=200/>|<img width=200/>|
|1| $\alpha_k = r_{k-1}^T r_{k-1}/d_{k}^T A d_{k}$ | Step length to next $x_k$ |
|2| $x_k = x_{k-1} + \alpha_k d_{k}$ | Approximate solution |
|3| $r_k = r_{k-1} - \alpha_k A d_k$ | New residual |
|4| $\beta_{k+1} = r_k^T r_k/ r_{k-1}^T r_{k-1}$ | Weight of $d_k$ for next direction |
|5| $d_{k+1} = r_k + \beta_{k+1}d_k$ | Next search direction |




However, for the non-linear problem like deep leaning, we need to reformulate the algorithm. There are two difference from the linear case.
1. Because $Ae_i$ is no longer equal to the residual $r_i$, or say negative gradient, we therefore need to compute the step size denoted by $\alpha$ using another method called line search.
2. For the reason that $\alpha$ has changed, $\beta$ can only be computed using its original form
$$\beta = \frac{r_i^T A d_i}{d_i^T A d_i}$$


## Determine the best step size (line search)

The main idea of *line search* is to find the point which has the gradient orthogonal to the current search direction.

$$g(W_{k+1}) = (W_{k+1}-W_{k})^T \nabla L(W_{k+1}) = cd_{k+1}^T r_{k+1} = 0$$

where we have defined the multiplication of the direction and the new gradient as a function $g(\cdot)$. Then our goal turns out to be finding the point that makes this function zero. We will introduce 3 methods here.

### Bisection method
1. Choose two points $a$ and $b$ on the search line that make $g(a) > 0$ and $g(b) < 0$.
2. Compute $g(c)$, where $c=\frac{a+b}{2}$ is the middle point of $a$ and $b$.
3. If the sign of $g(c) > 0$, which indicates that the zero point is in $(b, c)$, thus set $a = c$.
4. If the sign of $g(c) < 0$, which indicates that the zero point is in $(a, c)$, thus set $b = c$.
5. Run the steps $2$ to $4$ iteratively until convergence.


### Secant method
1. Choose two points $a$ and $b$ on the search line that make $g(a) > 0$ and $g(b) < 0$.
2. Compute $g(x)$, where $x$ is the intersection point of the line $g=0$ and the line that go through $g(a)$ and $g(b)$. $x$ therefore has the following expression.
$$\frac{g(b)-g(a)}{b-a}(x-a)+g(a)=0\Rightarrow x = a-\frac{b-a}{g(b)-g(a)}g(a)$$
3. If the sign of $g(x) > 0$, which indicates that the zero point is in $(b, x)$, thus set $a = x$.
4. If the sign of $g(x) < 0$, which indicates that the zero point is in $(a, x)$, thus set $b = x$.
5. Run the steps $2$ to $4$ iteratively until convergence.



### Dekker's method
Unlike the previous two methods, Dekker's method keeps track of three points:
- $a_k$: the current "contrapoint"
- $b_k$: the current guess for the root
- $b_{k-1}$: the previous guess for the root



1. Choose two points $a_0$ and $b_0$ on the search line that make $g(a_0) > 0$ and $g(b_0) < 0$, and define $g(b_{-1}) = g(a)$
2. Compute $c$ using Biection method with $a_k$ and $b_k$.
3. Compute $x$ using Secant method with $b_{k-1}$ and $b_k$.
4. If $c$ is between $x$ and $b_k$ then let $b_{k+1} = c$, otherwise let $b_{k+1} = x$.
5. Check convergence of $g(b_{k+1})$. Stop if convergent.
6. Choose $a_{k+1}$ from $a_k$ and $b_{k}$ such that it has the different sign with $b_{k+1}$.
7. If $g(b_{k+1}) > g(a_{k+1})$ then swap.
8. Run the steps $2$ to $7$ repeatively.



## <font color='#cccccc'>Determine the best direction (first-order)</font>
<font color='#cccccc'>
If we want to compute $\beta$ with only the first-order calculation, we can assume that we are solving the linear problem. Thus 

$$r_k-r_{k-1} = (b-Ax_k)-(b-Ax_{k-1}) = -A(x_k-x_{k-1}) = -cAd_k$$

Substitude into the euqation of $\beta$, we obtain

$$\beta = \frac{r_k^T A d_k}{d_k^T A d_k} = \frac{r_k^T(r_k - r_{k-1})}{d_k^T (r_k - r_{k-1})}$$

</font>


--------------

# Hessian-Free Optimization







----------------------

# Quasi-Newton Methods and BFGS







-----------------------

# Problems with Second-Order Methods:Saddle Points







In [51]:

# for showing iteratively
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
import warnings

# convert warnings to error
warnings.filterwarnings("ignore")

# learning rate decay
epoch_t = 0

# conjugate gradient algorithm
q0 = None

count = 0
H_count = 0

TRAIN_FINISHED = 100000

LT = 5000
LR = 0.1
X = None
Y = None
x = None
y = None
cont = None
levels = None

Gaussian = lambda t, mu, sigma: 1.0/(sigma*np.sqrt(2*np.pi))*np.exp(-(t-mu)**2/(2*sigma**2))

class EXP_TRAIN_FINISHED(Exception):
    pass

A = None
F = None
def Adam(derivatives):
    a0 = 0.1
    rf = 0.9
    r = 0.999
    global A, F
    if A is None:
        A = np.full(derivatives.shape, 0)
        F = np.full(derivatives.shape, 0)
    A = r*A + (1-r)*derivatives**2
    F = rf*F + (1-rf)*derivatives
    at = a0 * (np.sqrt(1-r)/(1-rf))
    return -at/(np.sqrt(A)+1e-8)*F

# def conjugate_gradient(derivatives, hessian):
#     global q0
#     if q0 is None:
#         q1 = -derivatives
#     else:
#         denominator = q0 @ hessian @ q0
#         if denominator == 0:
#             print(derivatives)
#             print(q0)
#             print(hessian)
#             raise Exception("conjugate gradient denominator zero")
#         numerator = q0 @ hessian @ derivatives
#         beta = numerator/denominator
#         q1 = -derivatives +  beta * q0
#     q0 = q1
#     alpha = (q0 @ derivatives) / (q0 @ hessian @ q0)
#     return alpha*q1

def conjugate_gradient(derivatives, last_derivatives, hessian, direction):
    residual = -derivatives
    last_residual = -last_derivatives
    if H_count != 1:
        beta = (residual @ residual)/(last_residual @ last_residual)
        direction = residual + beta * direction
    ret = 0.0
    denominator = direction @ hessian @ direction
    numerator = residual @ residual
    #numerator = direction @ direction
    if denominator == 0:
        print(direction)
        print(hessian)
        print(residual)
        raise Exception("conjugate gradient denominator zero")
    alpha = numerator/denominator
    #print(alpha)
    ret = alpha * direction
    return ret, direction

def tanh(a):
    y = np.tanh(a)
    return y

def dtanh(o):
    y = 1-o**2
    return y

def sigmoid(a):
    y = 1.0/(1.0+np.exp(-a))
    return y

def dsigmoid(o):
    y = o*(1-o)
    return y

def compute_weights_error(derivatives, variance):
    ret = LR * variance * derivatives
    return ret

class Layer:
    network = None
    unit_number = 0
    outputs = None
    errors = None
    weights = None
    derivatives = None
    prev_layer = None
    next_layer = None
    variance = 0.0
    
    w_start = 0
    b_start = 0
    act_errors = None
    
    act_func = tanh
    link_func = dtanh
    
    def __init__(self, K, afunc=tanh, lfunc=dtanh):
        self.unit_number = K
        self.outputs = np.zeros(K)
        self.errors = np.zeros(K)
        self.act_errors = np.zeros(K)
        self.act_func = afunc
        self.link_func = lfunc
        
    def build_connection(self, network):
        self.network = network
        j = self.prev_layer.unit_number
        self.variance = 1.0/(j * self.unit_number)
        
        w_size = self.unit_number * j
        self.w_start = self.network.w_size
        self.network.weights = np.hstack((self.network.weights, np.random.normal(0, np.sqrt(self.variance), w_size)))
        self.network.w_size += w_size
        
        b_size = self.unit_number
        self.b_start = self.network.w_size
        self.network.weights = np.hstack((self.network.weights, np.zeros(b_size)))
        self.network.w_size += b_size
        
    def forward_propagation(self):
        for k in range(self.unit_number):
            s = 0.0
            for j in range(self.prev_layer.unit_number):
                s += self.prev_layer.outputs[j] * self.network.weights[self.w_start+k*self.prev_layer.unit_number+j]
            s += self.network.weights[self.b_start + k]
            self.outputs[k] = self.act_func(s)

    def backward_propagation(self):
        for k in range(self.unit_number):
            s = 0.0
            a = 0.0
            for l in range(self.next_layer.unit_number):
                s += self.next_layer.errors[l] * self.network.weights[self.next_layer.w_start+l*self.unit_number+k]
                a += self.next_layer.act_errors[l] * self.network.weights[self.next_layer.w_start+l*self.unit_number+k]
            self.errors[k] = self.link_func(self.outputs[k]) * s
            self.act_errors[k] = self.link_func(self.outputs[k]) * a
            for j in range(self.prev_layer.unit_number):
                self.network.derivatives[self.w_start+k*self.prev_layer.unit_number+j] += self.errors[k] * self.prev_layer.outputs[j]
                self.network.b[self.w_start+k*self.prev_layer.unit_number+j] = self.act_errors[k] * self.prev_layer.outputs[j]
            self.network.derivatives[self.b_start+k] += self.errors[k]
            self.network.b[self.b_start+k] = self.act_errors[k]
            
    def dump(self):
        print("outputs {}".format(self.outputs))
        print("errors {}".format(self.errors))
        print("weights {}".format(self.weights))
        print("derivatives {}".format(self.derivatives))

# Mixture Gaussian Parameters Layer
class OutputLayer(Layer):
    targets = None
            
    def backward_propagation(self):
        self.errors = self.outputs-self.targets
        self.act_errors = np.ones(self.act_errors.shape)
        for k in range(self.unit_number):
            for j in range(self.prev_layer.unit_number):
                self.network.derivatives[self.w_start+k*self.prev_layer.unit_number+j] += self.errors[k] * self.prev_layer.outputs[j]
                self.network.b[self.w_start+k*self.prev_layer.unit_number+j] = self.act_errors[k] * self.prev_layer.outputs[j]
            self.network.derivatives[self.b_start+k] += self.errors[k]
            self.network.b[self.b_start+k] = self.act_errors[k]
            
    def update_target(self, t):
        self.targets = t

class InputLayer(Layer):
    outputs = None
    unit_number = 0
    prev_layer = None
    next_layer = None
    
    def __init__(self, K):
        self.unit_number = K
        
    def build_connection(self, network):
        self.network = network
        return
        
    def update_input(self, inputs):
        self.outputs = inputs

class Network:
    input_layer = None
    output_layer = None
    inputs = None
    targets = None
    hessian = None
    weights = None
    derivatives = None
    last_derivatives = None
    direction = None
    b = None
    w_size = 0
    observations = None
    targets = None
    
    def __init__(self):
        self.weights = np.array([])
        return
    
    def add_layer(self, layer):
        if self.input_layer == None:
            self.input_layer = layer
        else:
            layer.prev_layer = self.output_layer
            self.output_layer.next_layer = layer
            
        self.output_layer = layer
        layer.build_connection(self)
        
    def init_finished(self):
        self.derivatives = np.zeros(self.w_size)
        self.last_derivatives = np.zeros(self.w_size)
        self.hessian = np.zeros((self.w_size, self.w_size))
        self.b = np.zeros(self.w_size)
        
    def update_hessian(self):
        b = self.b.reshape(-1, 1)
        y = self.output_layer.outputs[0]
        self.hessian += y * (1-y) * (b @ b.T)
        
    def compute_finite_derivatives(self, observations, targets):
        N = len(observations)
        for n in range(N):
            self.input_layer.update_input(observations[n])
            self.output_layer.update_target(targets[n])
            
            layer = self.input_layer.next_layer
            while layer != None:
                layer.forward_propagation()
                layer = layer.next_layer
            layer = self.output_layer
            while layer != self.input_layer:
                layer.backward_propagation()
                layer = layer.prev_layer
    
    def compute_hessian_with_derivatives(self):
        e = 1e-8
        derivatives = np.copy(self.derivatives)
        length = len(self.weights)
        for i in range(length):
            error = np.zeros(self.derivatives.shape)
            self.weights[i] += e
            self.derivatives *= 0
            self.compute_finite_derivatives(self.observations, self.targets)
            error += self.derivatives
            self.weights[i] -= 2*e
            self.derivatives *= 0
            self.compute_finite_derivatives(self.observations, self.targets)
            error -= self.derivatives
            self.hessian[i] = error/(2*e)
            self.weights[i] += e
        self.derivatives = np.copy(derivatives)
        #print("compute with derivatives\n", self.hessian)
        
    def compute_hessian_with_error(self):
        e = 1e-5
        derivatives = np.copy(self.derivatives)
        length = len(self.weights)
        for i in range(length):
            for j in range(length):
                if j < i:
                    continue
                error = 0.0
                self.weights[i] += e
                self.weights[j] += e
                error += self.Get_Error(self.observations, self.targets, 0)
                self.weights[i] -= 2*e
                error -= self.Get_Error(self.observations, self.targets, 0)
                self.weights[i] += 2*e
                self.weights[j] -= 2*e
                error -= self.Get_Error(self.observations, self.targets, 0)
                self.weights[i] -= 2*e
                error += self.Get_Error(self.observations, self.targets, 0)
                self.weights[i] += e
                self.weights[j] += e
                self.hessian[i][j] = error/(4*e**2)
                self.hessian[j][i] = self.hessian[i][j]
                
        self.derivatives = np.copy(derivatives)
        #print("compute with error\n", self.hessian)
        #raise EXP_TRAIN_FINISHED("Test")
                
        
    def update(self, update_weight):
        #self.update_hessian()
        if update_weight == 1:
            if np.allclose(self.derivatives, np.zeros(self.w_size)):
                raise EXP_TRAIN_FINISHED("Train finished")
            
            # Adam
            global count, H_count
            if count<0:
                count += 1
                ret = Adam(self.derivatives)
            else:
            # Conjugate Gradients algorithm
                #self.compute_hessian_with_error()
                if H_count % 1 == 0:
                    self.compute_hessian_with_derivatives()
                H_count += 1
                if self.direction is None:
                    self.direction = -np.copy(self.derivatives)
                ret, self.direction = conjugate_gradient(self.derivatives, self.last_derivatives, self.hessian, self.direction)
                self.last_derivatives = np.copy(self.derivatives)
                #ret = (-np.linalg.inv(self.hessian) @ self.derivatives)
            
            self.weights += ret
            print("weight is \n", self.weights)
            self.derivatives *= 0
            #self.hessian *= 0
        return 1
        
    def train(self, observations, targets):
        self.observations = observations
        self.targets = targets
        
        N = len(observations)
        for n in range(N):
            self.input_layer.update_input(observations[n])
            self.output_layer.update_target(targets[n])
            
            layer = self.input_layer.next_layer
            while layer != None:
                layer.forward_propagation()
                layer = layer.next_layer
            
            layer = self.output_layer
            while layer != self.input_layer:
                layer.backward_propagation()
                layer = layer.prev_layer
            
            # gradient descent/stotastic gradient descent
            if n == N-1:
                update_weight = 1
            else:
                update_weight = 0
            self.update(update_weight)

    def Get_Error(self, observations, targets, show):
        N = len(observations)
        es = 0.0
        for n in range(N):
            self.input_layer.update_input(observations[n])
            self.output_layer.update_target(targets[n])
            
            layer = self.input_layer.next_layer
            while layer != None:
                layer.forward_propagation()
                layer = layer.next_layer
            #print("target {}".format(self.targets))
            #print("output {}".format(self.output_layer.outputs))
            #e = np.sum(self.targets * np.log(self.output_layer.outputs))
            # for sigmoid only
            if targets[n] == 0:
                e = np.log(1 - self.output_layer.outputs[0])
            else:
                e = np.log(self.output_layer.outputs[0])
            es += e
            if show == 1:
                print(self.targets, self.output_layer.outputs)
        return -es
                
    def test(self, new_input):
        self.input_layer.update_input(new_input)
        
        layer = self.input_layer.next_layer
        while layer != None:
            layer.forward_propagation()
            layer = layer.next_layer
        return self.output_layer.outputs
    
    def dump(self):
        layer = self.input_layer.next_layer
        while layer != None:
            layer.dump()
            layer = layer.next_layer
        return

def gen_training_data(ax):
    mean1 = [0.2, 0.2]
    cov1 = [[0.03, -0.01], 
           [-0.01, 0.02]]
    X1 = np.random.multivariate_normal(mean1, cov1, 20)

    mean12 = [0.7, 0.7]
    cov12 = [[0.01, -0.01], 
           [-0.01, 0.02]]
    X12 = np.random.multivariate_normal(mean12, cov12, 20)
    X1 = np.vstack((X1, X12))
    T1 = np.zeros(len(X1))
    
    mean2 = [0.4, 0.6]
    cov2 = [[0.03, -0.02], 
           [-0.02, 0.04]]
    X2 = np.random.multivariate_normal(mean2, cov2, 40)
    T2 = np.ones(len(X2))
    
    ax.scatter(X1.T[0], X1.T[1], s=50,  facecolors='none', edgecolors='blue')
    ax.scatter(X2.T[0], X2.T[1], s=50,  facecolors='red', edgecolors='none', marker='x')
    
    X = np.vstack((X1, X2))
    T = np.hstack((T1, T2))
    return X, T

def show_pic(network, ax, fig, init=0):
    global x, y, X, Y, Z, cont, levels
    if init == 1:
        x = np.linspace(0,1,20)
        y = np.linspace(0,1,20)
        X, Y = np.meshgrid(x, y)
        levels = np.arange(0.1, 1.1, 0.2)
    
    Z = np.zeros(X.shape)
    for i in range(len(y)):
        for j in range(len(x)):
            Z[i][j] = network.test([x[j], y[i]])
    if init != 1:
        for coll in cont.collections: 
            coll.remove()
    cont = ax.contour(X, Y, Z, levels=levels) 
    fig.canvas.draw()
    
def training_show_process(network, X, T, fig, ax):
    show_pic(network, ax, fig, 1)

    for i in range(LT):
        network.train(X, T)
        if i % 10 != 9:
            show_pic(network, ax, fig)
    return

def build_network():
    network = Network()
    ilayer = InputLayer(2)
    network.add_layer(ilayer)
    layer1 = Layer(3)
    network.add_layer(layer1)
    # layer2 = Layer(5)
    # network.add_layer(layer2)
    olayer = OutputLayer(1, sigmoid, dsigmoid)
    network.add_layer(olayer)
    network.init_finished()
    return network

def main():
    fig = plt.figure(figsize=(11,5), dpi=60)
    ax1 = fig.add_subplot(1,2,1)
    ax2 = fig.add_subplot(1,2,2)
    ax1.set_xlim(0, 1)
    ax1.set_ylim(0, 1)
    
    X, T = gen_training_data(ax1)
    network = build_network()
    try:
        training_show_process(network, X, T, fig, ax1)
    except EXP_TRAIN_FINISHED:
        show_pic(network, ax1, fig)
        raise

if __name__=="__main__":
    main()

<IPython.core.display.Javascript object>

weight is 
 [ 0.50559373  0.27075718  0.023369   -0.50602523 -0.24940212  0.16087654
 -0.09043584 -0.01006336  0.07920559 -0.56118914 -0.18673203  0.45352508
  0.17335233]
weight is 
 [ 1.85671696 -6.34229072  0.82614094 -1.87681429 -2.30110702  4.77590556
  0.94056265  1.13485737 -1.45488255  1.34071215 -5.21602781  2.46442282
 -2.92421694]
weight is 
 [  2.6793926  -10.35336411   1.27747334  -2.77675917  -3.53487514
   7.59321545   1.57200722   1.72332597  -2.35657171   2.474997
  -8.26195395   3.69403217  -4.77943092]
weight is 
 [ 15.7372448  -73.87964839   8.03374023 -17.84708068 -22.99285663
  52.3493483   11.62623731   9.83217476 -16.41619395  20.26729748
 -56.44279125  23.27030872 -33.96565213]
weight is 
 [ 18.98144096 -89.60717248   9.60850145 -21.78140602 -27.80074416
  63.45030106  14.13911978  11.53151102 -19.86273144  24.64840083
 -68.34084272  28.12750239 -41.16501638]
weight is 
 [  22.67560491 -107.45541479   11.27784982  -26.46358866  -33.24963139
   76.06618599   17.

weight is 
 [  21.95781927 -103.90423187   10.82440574  -25.7810235   -32.15603818
   73.58011012   16.4801299    12.44456059  -22.93582224   28.58259253
  -79.08548886   32.56196916  -47.65581928]
weight is 
 [  21.96770827 -103.94918002   10.82013153  -25.80628446  -32.1695194
   73.6125759    16.48859552   12.42741319  -22.94459638   28.59437704
  -79.11830542   32.576006    -47.67555436]
weight is 
 [  21.97142963 -103.966075     10.81844815  -25.81589578  -32.17458501
   73.62478402   16.49178652   12.42078874  -22.94788729   28.59880157
  -79.1306328    32.58128319  -47.68296662]
weight is 
 [  21.97431441 -103.97915602   10.81708457  -25.82342941  -32.17850571
   73.63424025   16.49426445   12.41551707  -22.95042953   28.60222329
  -79.14017114   32.58536987  -47.68870104]
weight is 
 [  21.977465   -103.99342429   10.81553091  -25.83174902  -32.18278071
   73.64455932   16.49697558   12.40960764  -22.95319596   28.60595111
  -79.15056813   32.58982829  -47.69495081]
weight is 


weight is 
 [  22.3413123  -105.60314631   10.50817428  -26.96864046  -32.66190166
   74.81826981   16.82031716   11.42420544  -23.25181411   29.01907061
  -80.30975157   33.09282164  -48.39126226]
weight is 
 [  22.34174868 -105.60491352   10.50760636  -26.97031332  -32.66241473
   74.81959859   16.82074896   11.42231071  -23.25208618   29.01952506
  -80.31097013   33.09334313  -48.39202221]
weight is 
 [  22.34221724 -105.60680151   10.50698763  -26.97212431  -32.66296204
   74.82102074   16.82121518   11.42023894  -23.25237331   29.02001085
  -80.31226856   33.09389808  -48.39283404]
weight is 
 [  22.34273635 -105.6088823    10.5062925   -26.97414688  -32.6635643
   74.82259104   16.82173461   11.41790205  -23.25268572   29.02054661
  -80.31369567   33.09450721  -48.39372873]
weight is 
 [  22.34333745 -105.61127883   10.50547665  -26.97650753  -32.66425683
   74.82440312   16.82233958   11.4151476   -23.25304069   29.0211641
  -80.31533466   33.09520575  -48.39475916]
weight is 
 

weight is 
 [  22.34056051 -105.60071416   10.50930439  -26.96530454  -32.66124785
   74.81627741   16.81940838   11.42912927  -23.25166572   29.01840961
  -80.30829909   33.09225953  -48.39020261]
weight is 
 [  22.34070062 -105.60119024   10.50902329  -26.96599829  -32.66137823
   74.81665983   16.8195716    11.42814111  -23.25170518   29.01853434
  -80.30859498   33.09237968  -48.39040637]
weight is 
 [  22.34084196 -105.60166858   10.50873767  -26.9667014   -32.66150904
   74.81704469   16.81973677   11.42713634  -23.25174399   29.01865972
  -80.30889147   33.0924999   -48.39061107]
weight is 
 [  22.34098528 -105.6021517    10.50844598  -26.96741771  -32.66164095
   74.81743403   16.8199048    11.42610937  -23.25178231   29.01878641
  -80.3091901    33.09262079  -48.3908178 ]
weight is 
 [  22.34113145 -105.60264238   10.50814644  -26.96815154  -32.66177472
   74.8178301    16.82007671   11.42505384  -23.25182033   29.01891515
  -80.30949252   33.09274304  -48.39102773]
weight is 

...

weight is 
 [  23.05153848 -105.27864872   12.11949967  -27.15627468  -32.22875097
   75.51777769   18.38493744    8.22240793  -21.81933414   29.53453969
  -78.82808122   31.8271736   -48.73308948]
weight is 
 [  23.05157251 -105.27862973   12.11958541  -27.15625318  -32.22872537
   75.51781384   18.38501332    8.22231358  -21.8192534    29.53457336
  -78.82800158   31.82710253  -48.73311224]
weight is 
 [  23.05160657 -105.27861071   12.11967127  -27.15623163  -32.22869974
   75.51785004   18.38508929    8.22221914  -21.81917253   29.53460709
  -78.82792182   31.82703134  -48.73313505]
weight is 
 [  23.05164069 -105.27859167   12.11975727  -27.15621002  -32.22867406
   75.51788629   18.38516537    8.22212463  -21.81909155   29.53464088
  -78.82784194   31.82696004  -48.73315789]
weight is 
 [  23.05167485 -105.2785726    12.11984339  -27.15618836  -32.22864835
   75.5179226    18.38524156    8.22203003  -21.81901044   29.53467473
  -78.82776193   31.82688862  -48.73318078]
weight is 

weight is 
 [  23.0530113  -105.27782653   12.1232231   -27.15532149  -32.22763874
   75.51934784   18.38822222    8.21836587  -21.81582622   29.53600949
  -78.82461694   31.82407932  -48.73408393]
weight is 
 [  23.0530476  -105.27780626   12.12331517  -27.15529743  -32.22761122
   75.51938668   18.38830318    8.21826732  -21.81573945   29.53604602
  -78.82453113   31.82400262  -48.73410866]
weight is 
 [  23.05308397 -105.27778596   12.12340741  -27.15527331  -32.22758365
   75.51942559   18.38838428    8.21816866  -21.81565251   29.53608263
  -78.82444516   31.82392576  -48.73413345]
weight is 
 [  23.0531204  -105.27776562   12.12349983  -27.15524911  -32.22755603
   75.51946457   18.38846553    8.21806986  -21.8155654    29.53611931
  -78.82435901   31.82384874  -48.73415829]
weight is 
 [  23.05315689 -105.27774524   12.12359243  -27.15522485  -32.22752835
   75.51950363   18.38854692    8.21797094  -21.81547812   29.53615608
  -78.82427269   31.82377157  -48.73418319]
weight is 

weight is 
 [  23.05459789 -105.27694068   12.12725933  -27.15424623  -32.22643182
   75.52105071   18.39176077    8.21410472  -21.81202083   29.53761843
  -78.82084923   31.82070876  -48.73517422]
weight is 
 [  23.05463739 -105.27691862   12.12736014  -27.15421886  -32.22640166
   75.52109325   18.39184888    8.21399977  -21.81192575   29.5376588
  -78.82075497   31.82062437  -48.7352016 ]
weight is 
 [  23.05467699 -105.2768965    12.1274612   -27.15419139  -32.22637142
   75.5211359    18.3919372     8.21389465  -21.81183044   29.53769928
  -78.82066048   31.82053978  -48.73522906]
weight is 
 [  23.05471668 -105.27687434   12.1275625   -27.15416383  -32.22634112
   75.52117865   18.39202571    8.21378934  -21.8117349    29.53773987
  -78.82056575   31.82045496  -48.73525659]
weight is 
 [  23.05475646 -105.27685213   12.12766406  -27.15413618  -32.22631073
   75.5212215    18.39211444    8.21368384  -21.81163912   29.53778057
  -78.82047079   31.82036993  -48.73528419]
weight is 


weight is 
 [  23.05634413 -105.27596553   12.13172822  -27.1530105   -32.22509432
   75.5229368    18.39565545    8.20951685  -21.80780502   29.53941616
  -78.81666475   31.81695987  -48.73639426]
weight is 
 [  23.05638814 -105.27594094   12.13184118  -27.1529787   -32.2250605
   75.52298448   18.39575361    8.2094025   -21.80769842   29.5394618
  -78.81655882   31.8168649   -48.73642525]
weight is 
 [  23.05643229 -105.27591629   12.13195448  -27.15294678  -32.22502657
   75.52303231   18.39585206    8.20928788  -21.80759151   29.53950759
  -78.81645256   31.81676964  -48.73645635]
weight is 
 [  23.05647656 -105.27589157   12.13206813  -27.15291473  -32.22499254
   75.52308028   18.3959508     8.20917299  -21.80748426   29.53955352
  -78.81634597   31.81667407  -48.73648755]
weight is 
 [  23.05652096 -105.27586677   12.13218213  -27.15288256  -32.22495841
   75.52312841   18.39604983    8.20905782  -21.80737669   29.53959961
  -78.81623905   31.8165782   -48.73651885]
weight is 
 

weight is 
 [  23.0583167  -105.27486382   12.13680488  -27.15155677  -32.22357377
   75.52507995   18.40005495    8.20444903  -21.8030136    29.54147589
  -78.81189741   31.81268294  -48.73779403]
